In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

In [4]:
for layer in model_vgg16_conv.layers:
    layer.trainable = False

In [11]:
img_width, img_height = 150, 150
train_data_dir = "C:/Users/DELL/Downloads/CNN_letter dataset/all_letters_image"
labels_csv_file = "C:/Users/DELL/Downloads/CNN_letter dataset/all_letters_info.csv"
model_weights_file = 'vgg16-xfer-weights.h5'
nb_train_samples = 4
nb_epochs = 5

In [12]:
input = Input(shape=(img_width, img_height, 3))
output_vgg16_conv = model_vgg16_conv(input)
x = Flatten()(output_vgg16_conv)
x = Dense(64, activation='relu')(x)
x = Dense(2, activation='softmax')(x)
model = Model(input, x)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [14]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
#augment  images
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height),
                                                    batch_size=4, class_mode='categorical')


Found 14191 images belonging to 1 classes.


In [22]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [24]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, epochs=5, callbacks = callbacks)
print('Training Completed!')

Epoch 1/5


3548/3548 [==============================] - 955s 269ms/step - loss: 3202458.2500 - accuracy: 0.0000e+00
Epoch 2/5
3548/3548 [==============================] - 944s 266ms/step - loss: 9201183.0000 - accuracy: 0.0000e+00
Epoch 3/5
3548/3548 [==============================] - 1124s 317ms/step - loss: 14747589.0000 - accuracy: 0.0000e+00
Epoch 4/5
3548/3548 [==============================] - 1124s 317ms/step - loss: 20123540.0000 - accuracy: 0.0000e+00
Epoch 5/5
3548/3548 [==============================] - 940s 265ms/step - loss: 25253388.0000 - accuracy: 0.0000e+00
Training Completed!


In [25]:
labels_df = pd.read_csv(labels_csv_file)

In [26]:
labels_df.set_index('letter', inplace=True)

In [27]:
input = Input(shape=(img_width, img_height, 3))
output_vgg16_conv = model_vgg16_conv(input)
x = Flatten()(output_vgg16_conv)
x = Dense(64, activation='relu')(x)
x = Dense(len(labels_df), activation='softmax')(x)  # Update the output layer to match the number of labels
model = Model(input, x)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [28]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height),
                                                    batch_size=4, class_mode='categorical')


Found 14191 images belonging to 1 classes.


In [31]:
img_path ="C:/Users/DELL/Downloads/CNN_letter dataset/all_letters_image/all_letters_image/01_07.png"

img = image.load_img(img_path, target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Predict the class probabilities
features = model.predict(x)[0]

# Get the predicted label
predicted_class_index = np.argmax(features)
predicted_label = labels_df.index[predicted_class_index]

print('Predicted Array:', features)
print('Predicted Label:', predicted_label)

1/1 [==============================] - 0s 168ms/step
Predicted Array: [1.0647052e-04 5.9522092e-05 4.8125905e-05 ... 8.5729902e-05 1.2700380e-04
 4.8197482e-05]
Predicted Label: а
